In [1]:
import sys
import pandas as pd
import numpy as np

sys.path.append("../")
import biobss
import matplotlib.pyplot as plt
import neurokit2 as nk
from plotly_resampler import register_plotly_resampler

In [2]:
import importlib
import sys
import pandas as pd
import numpy as np

sys.path.append("../")
import biobss
import os
os.getcwd()
import neurokit2 as nk
sample_data = biobss.utils.load_sample_data("ECG")

In [3]:
sr=256

In [4]:
sample=np.array(sample_data[0]).flatten()
pipeline = biobss.pipeline.Bio_Pipeline(windowed_process=False)

In [5]:
clean=biobss.pipeline.Bio_Process(
    process_method=biobss.ecgtools.filter_ecg,process_name="clean",argmap={"sampling_rate":"sampling_rate"},method="pantompkins")

find_peaks = biobss.pipeline.Bio_Process(process_method=biobss.ecgtools.ecg_peaks.ecg_detectpeaks,
                                         process_name="rpeaks",returns_event=True,event_args={"event_name":"ECG_Peaks"})




pipeline.process_queue.add_process(clean,input_signals="ECG_Raw",output_signals="ECG_Clean")
pipeline.process_queue.add_process(find_peaks,input_signals="ECG_Clean",output_signals="ECG_Peaks")

channel=biobss.pipeline.Bio_Channel(signal=sample,name="ECG_Raw",sampling_rate=sr)
pipeline.set_input(channel)


In [6]:
pipeline

Bio_Pipeline:
	Preprocessors: Process list:

	Processors: Process list:
	1: clean(ECG_Raw) -> ECG_Clean
	2: rpeaks(ECG_Clean) -> ECG_Peaks

	Postprocessors: Process list:

	Window Size(Seconds): Not Windowed
	Step Size: Not Windowed

In [7]:
# Create feature extraction steps

# Extract time domain features. input_signals dictionary contains feature prefixes as keys and input signals as values
# For example rms of EDA_Tonic is extracted as Tonic_rms in this case
Rpeak_features = biobss.pipeline.Feature(name="time_features", function=biobss.ecgtools.from_Rpeaks, sampling_rate=sr)
stat_features = biobss.pipeline.Feature(name="stat_features", function=biobss.edatools.eda_stat_features)

In [8]:
pipeline.run_pipeline()

In [9]:
pipeline.add_feature_step(Rpeak_features,input_signals=['ECG_Raw','Events_ECG_Clean'])
pipeline.add_feature_step(stat_features,input_signals=['ECG_Clean'],prefix="ECG")

In [10]:
pipeline

Bio_Pipeline:
	Preprocessors: Process list:

	Processors: Process list:
	1: clean(ECG_Raw) -> ECG_Clean
	2: rpeaks(ECG_Clean) -> ECG_Peaks

	Postprocessors: Process list:

	Window Size(Seconds): Not Windowed
	Step Size: Not Windowed

In [11]:
pipeline.data['Events_ECG_Clean'].timestamp_data

array([ 359.375  , 1007.8125 , 1644.53125, 2285.15625, 2957.03125,
       3613.28125, 4261.71875, 4898.4375 , 5539.0625 , 6183.59375,
       6839.84375, 7472.65625, 8113.28125, 8761.71875, 9425.78125])

In [12]:
pipeline.data['Events_ECG_Clean'].window_timestamps

[359.375, 9425.78125]

In [13]:
pipeline.extract_features()

c:\Users\cagatay.tasci\Desktop\Biobss\conflict\BIOBSS\examples\..\biobss\pipeline\feature_queue.py:79: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  timestamps = np.array(timestamps)


ValueError: Timestamps must be the same for all input signals.

In [14]:
pipeline.features

,ecg_a_R,ecg_RR0,ecg_RR1,ecg_RR2,ecg_RRm,ecg_RR_0_1,ecg_RR_2_1,ecg_RR_m_1,ECG_mean,ECG_std,ECG_max,ECG_min,ECG_range,ECG_kurtosis,ECG_skew,ECG_momentum
0.0,"[0.372, 0.454, 0.125, 0.249, 0.134]","[0.6484375, 0.63671875, 0.640625, 0.671875, 0....","[0.63671875, 0.640625, 0.671875, 0.65625, 0.64...","[0.640625, 0.671875, 0.65625, 0.6484375, 0.636...","[0.6419270833333334, 0.6497395833333334, 0.656...","[1.01840490797546, 0.9939024390243902, 0.95348...","[1.0061349693251533, 1.048780487804878, 0.9767...","[1.0081799591002045, 1.014227642276423, 0.9767...",-0.000339,0.053742,0.221432,-0.132590,0.354022,1.986408,0.758343,0.002888
1000.0,"[0.454, 0.125, 0.249, 0.134, 0.378]","[0.3125, 0.640625, 0.671875, 0.65625, 0.6484375]","[0.640625, 0.671875, 0.65625, 0.6484375, 0.636...","[0.671875, 0.65625, 0.6484375, 0.63671875, 0.6...","[0.5416666666666666, 0.65625, 0.65885416666666...","[0.4878048780487805, 0.9534883720930233, 1.023...","[1.048780487804878, 0.9767441860465116, 0.9880...","[0.8455284552845528, 0.9767441860465116, 1.003...",0.000160,0.051982,0.221432,-0.132590,0.354022,1.980885,0.705849,0.002702
2000.0,"[-0.064, 0.134, 0.378, 0.389, 0.222]","[0.5625, 0.7265625, 0.6484375, 0.63671875, 0.6...","[0.7265625, 0.6484375, 0.63671875, 0.640625, 0...","[0.6484375, 0.63671875, 0.640625, 0.64453125, ...","[0.6458333333333334, 0.6705729166666666, 0.641...","[0.7741935483870968, 1.1204819277108433, 1.018...","[0.8924731182795699, 0.9819277108433735, 1.006...","[0.888888888888889, 1.034136546184739, 1.00817...",-0.000032,0.052829,0.221432,-0.132590,0.354022,1.901770,0.706685,0.002791
3000.0,"[-0.217, -0.063, 0.389, 0.222, 0.175]","[0.32421875, 0.53515625, 0.70703125, 0.640625,...","[0.53515625, 0.70703125, 0.640625, 0.64453125,...","[0.70703125, 0.640625, 0.64453125, 0.65625, 0....","[0.5221354166666666, 0.6276041666666666, 0.664...","[0.6058394160583942, 0.7569060773480663, 1.103...","[1.3211678832116789, 0.9060773480662984, 1.006...","[0.9756690997566909, 0.8876611418047882, 1.036...",0.000331,0.051645,0.221432,-0.128796,0.350229,2.211473,0.763516,0.002667
4000.0,"[-0.077, 0.222, 0.175, 0.436, 0.343]","[0.51953125, 0.71484375, 0.64453125, 0.65625, ...","[0.71484375, 0.64453125, 0.65625, 0.6328125, 0...","[0.64453125, 0.65625, 0.6328125, 0.640625, 0.6...","[0.6263020833333334, 0.671875, 0.64453125, 0.6...","[0.726775956284153, 1.1090909090909091, 0.9821...","[0.9016393442622951, 1.018181818181818, 0.9642...","[0.8761384335154827, 1.0424242424242425, 0.982...",0.000044,0.053749,0.221431,-0.133306,0.354736,1.984512,0.731601,0.002889
5000.0,"[-0.007, 0.175, 0.436, 0.343, 0.15]","[0.3046875, 0.5546875, 0.65625, 0.6328125, 0.6...","[0.5546875, 0.65625, 0.6328125, 0.640625, 0.64...","[0.65625, 0.6328125, 0.640625, 0.6484375, 0.66...","[0.5052083333333334, 0.6145833333333334, 0.643...","[0.5492957746478874, 0.8452380952380952, 1.037...","[1.1830985915492958, 0.9642857142857143, 1.012...","[0.9107981220657277, 0.9365079365079365, 1.016...",0.000474,0.051075,0.204114,-0.133314,0.337428,1.879659,0.659768,0.002609
